In [1]:
from threeDprediction import *
from threeDprediction_multi import *
from pore_analysis import *
from visualization import *
import os
import numpy as np
from skimage import io
import tensorflow

print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

Num GPUs Available:  0


# Welcome to the Jupyter Notebook for the utilization of UTILE-Pore

With this notebook, you will be guided through the code and make it possible for you to analyze your porous materials tomographies automatically.

This notebook should help you to run the automated segmentation of the volumes in your images and afterward to apply the diverse functions to extract the information of interest from your data and visualize the results.

Already integrated functions are:

GDL/porous material
- Pore size distribution analysis of the pores with PoreSpy
- Surface roughness calculation
- Tortuosity simulation with PoreSpy
- Permeability estimation using Kozeny-Carman equation
- Calculation of the solid surface ratio


For MPL
- Layer thickness calculation
- MPL crack analsis (ratio, size distribution, etc.)
- MPL heatmap for thickness variation
- MPL intrusion measurement
- MPL and GDL contact surface calculation

In [2]:
#Give a name to your project

case_name = "Practice2"

#First, we need to specify the folder where the tomograph slices (or tif stack) are stored

image_path= "C:/Users/andre/Downloads/39bb2_HRNET_HRNET_multi_fusev2_dataset_noaug_-0.6808-23.keras (1).tif"

#MPL?
mpl = False
#Secondly, we need to specify where is your model stored
if mpl == True:
    model_path = "C:/Users/andre/Desktop/Julich/pore/HRNET_HRNET_multi_fusev2_dataset_noaug_-0.6808-23.keras"
    #is the mpl in the top or the bottom of your stack?
    mpl_place = 'top'
    
else:
    model_path = "C:/Users/TEAM-CT/Desktop/UTILE-Pore/models/HRNET_HRNET_fusev2_utile_noaug_-0.5653-12.keras"

#It is also required to create a folder to store the predicted masks
os.makedirs(f"./{case_name}/", exist_ok=True)

# We set the path to the csv file to store all the extracted data
csv_file = f'./{case_name}/csv_{case_name}.csv'

In [3]:
# Now we start with the segmentation of the volume
# If you just have a porous materials and need a binary segmentation, run the following lines

# input_volume = io.imread('/p/project1/claimd/Andre/Aimy/Dataset/cal_fusev2/test/Toray1202.tif')
# predicted_volume = process_and_predict(image_path, model_path)
# io.imsave(f'./predicted_{case_name}.tif', predicted_volume.astype(np.uint8))

#If you are analyzing a GDL with MPL, then you can run a multiclass semantic segmentation with the following lines:
input_volume = io.imread(image_path)
if mpl:
    if mpl_place != 'top':
         input_volume = np.flip(input_volume, axis=0)
    predicted_volume = process_and_predict_multi(input_volume, model_path)
else:
    predicted_volume = process_and_predict(input_volume, model_path)
io.imsave(f'./{case_name}/predicted_{case_name}.tif', predicted_volume.astype(np.uint8))

OSError: No file or directory found at C:/Users/TEAM-CT/Desktop/UTILE-Pore/models/HRNET_HRNET_fusev2_utile_noaug_-0.5653-12.keras

In [6]:
# Now, lets visualize the segmented volume
predicted_volume = io.imread(image_path)
visualize_volume(predicted_volume, case_name, False)

In [6]:
# We can directly start wiht the property extraction of the porous material or GDL
porosity, results, avg_pore, sd = calculate_psd(predicted_volume, csv_file, case_name, voxel_size=1)

Porosity: 0.6344364417231323


  0%|          | 0/25 [00:00<?, ?it/s]

Sizes shape: (410, 410, 410)
Results: ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Results of pore_size_distribution generated at Thu Mar 27 17:12:10 2025
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
LogR                      Array of size (10,)
pdf                       Array of size (10,)
cdf                       Array of size (10,)
satn                      Array of size (10,)
bin_centers               Array of size (10,)
bin_edges                 Array of size (11,)
bin_widths                Array of size (10,)
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Average Pore Size (voxels): 0.6323549063522945
Standard Deviation (voxels): 0.5501288380830935


In [5]:
#Tortuosity simulation (for non-optimized devices can take several hours)
tortuosity = 1.85
#tortuosity = tortuosity_simulation(predicted_volume, csv_file)

In [6]:
# Having the above mentioned properties, we can estimate the permeability
# Estimate permeability
ssa = calculate_ssa(predicted_volume)
print('SSA: ',ssa)
porosity = 0.82
k_constant = 50
permeability = estimate_permeability(porosity, k_constant, csv_file, ssa, tortuosity)
print(f'Permeability: {permeability}')

Surface Area: 228195540.8181501
Solid Volume: 1926707750.0
SSA:  0.11843806660255043
Permeability: 7.089252862092315


In [7]:
# Then surface roughness calculation of the material
#Calculate surface roughness
Ra, Rq = calculate_surface_roughness_from_surface(predicted_volume)
with open(csv_file, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write data
        writer.writerow(['##### Surface roughness #####'])
        writer.writerow(['Arithmetic_Mean_Roughness_(Ra)', Ra])
        writer.writerow(['Root_Mean_Square_Roughness_(Rq)', Rq])
print(f"Arithmetic Mean Roughness (Ra): {Ra}")
print(f"Root Mean Square Roughness (Rq): {Rq}")


[1. 0. 0. ... 9. 9. 8.]
Arithmetic Mean Roughness (Ra): 11.329303979873657
Root Mean Square Roughness (Rq): 13.25063705444336


In [8]:
#Calculacte solid surface ratio
calculate_solid_surface_ratio(predicted_volume, csv_file, gdl=1, side='bottom')

Solid surface ratio: 0.23308763440428057


0.23308763440428057

In [15]:
# Now we can move to the functions specialized for MPL containing GDLs
# We can caluclate the MPL and GDL thickness
MPL_GDL_thickness(predicted_volume, csv_file, axis=0, mpl=2, gdl=1)


MPL avg Thickness 4  voxels
GDL avg Thickness 7  voxels
Max GDL Thickness 200  microns Max MPL Thickness 90  microns


In [19]:
#Calculate MPL crack analysis
crack_ratio, crack_count, crack_labels, crack_sizes, slice_image = MPL_crack_analysis(predicted_volume,case_name, csv_file, from_top=True)
print(f"Crack Ratio: {crack_ratio}")
print(f"Crack Count: {crack_count}")
#print(f"Crack Sizes: {crack_sizes}")
plot_crack_labels(slice_image, case_name, crack_labels)


Crack Ratio: 0.29280567859417517
Crack Count: 378


In [7]:
# Example usage:
Ra, Rq, Ra_std_dev, Ra_CoV, avg_thickness = MPL_intrusion_roughness(predicted_volume, csv_file, mpl=2, voxel_size=5, from_top=True)
print(f"Global Roughness Ra: {Ra}, Rq: {Rq}")
print(f"Standard Deviation of Local Ra: {Ra_std_dev}")
print(f"Coefficient of Variation of Local Ra: {Ra_CoV}")
print(f"Average Thickness: {avg_thickness}")

MPL Thickness 30
[0. 0. 0. ... 3. 3. 3.]
Global Roughness Ra: 4.78726863861084, Rq: 5.517195463180542
Standard Deviation of Local Ra: 0.4905248542311615
Coefficient of Variation of Local Ra: 0.1024644512895982
Average Thickness: 6


In [22]:
# MPL heatmap creation
MPL_heatmap(predicted_volume,case_name, 2)

In [13]:
# Calculate fiber touching MPL voxels
MPL_count_touching_voxels(predicted_volume, csv_file, mpl_class=2, fiber_class=1)

MPL voxels touching GDL:  789217


789217

In [21]:
# Calculate the pores and throats of the pore structure with the Snow algorithm and save them as a pickle file
if mpl == True:
    predicted_volume_snow = np.where(predicted_volume==1, 1,0) #Do this to remove the mpl and do the analysis only on the GDL
else: predicted_volume_snow = predicted_volume
snow_network_from_image(predicted_volume_snow, case_name)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[10:31:34] WARNING  Image was cropped to (46, 1802, 1034) during watershed                            ]8;id=63019;file://C:\Users\a.colliard\Miniconda3\envs\label\Lib\site-packages\porespy\networks\_snow2.py\_snow2.py]8;;\:]8;id=431256;file://C:\Users\a.colliard\Miniconda3\envs\label\Lib\site-packages\porespy\networks\_snow2.py#184\184]8;;\

Extracting pore and throat properties:   0%|          | 0/24988 [00:00<?, ?it/s]

snow_output saved to ./Practice/snow_Practice.pkl
